In [1]:
import scrapy
import pandas
import json
from scrapy.crawler import CrawlerProcess
import csv

class OlxCrawler(scrapy.Spider):
    name = 'olx'
    
    header = {"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 OPR/89.0.4447.48"}
    url = "https://www.olx.in/api/relevance/v2/search?category=1725&facet_limit=100&lang=en-IN&location=1000001&location_facet_limit=20&platform=web-desktop&size=40"
    def start_requests(self):
        for i in range(5):
            yield scrapy.Request(url = self.url + f"&page={i}", headers = self.header, callback = self.parse)
    
    def __init__(self):
        with open('olxdata.csv','w') as f:
            f.write('title, description, features, location, date_listed,price\n')
    def parse(self,request):
        html = request.text
        data = json.loads(html)
        for i in data['data']:
#             print(json.dumps(i,indent = 2))
            item = {
                'title' : i['title'],
                'description' : i['description'].strip().replace("\n"," "),
                'features': i['main_info'],
                'location':i['locations_resolved']['SUBLOCALITY_LEVEL_1_name']+", " + i['locations_resolved']\
                ["ADMIN_LEVEL_3_name"]+", " + i['locations_resolved']["ADMIN_LEVEL_1_name"]\
                +", " + i['locations_resolved']["COUNTRY_name"],
                'date_listed' : i['display_date'],
                'price' : i['price']['value']['display']
            }
#             print(json.dumps(item, indent=2))
            with open("olxdata.csv",'a') as file:
                writer = csv.DictWriter(file, fieldnames=item.keys())
                writer.writerow(item)

process = CrawlerProcess()
process.crawl(OlxCrawler)
process.start()

# OlxCrawler.parse(OlxCrawler,'')

2022-07-24 21:49:29 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-07-24 21:49:29 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 13:09:58) - [GCC 7.5.0], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Linux-5.15.0-41-generic-x86_64-with-glibc2.35
2022-07-24 21:49:29 [scrapy.crawler] INFO: Overridden settings:
{}
2022-07-24 21:49:29 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-07-24 21:49:29 [scrapy.extensions.telnet] INFO: Telnet Password: 84ff8ec54a3c4f65
2022-07-24 21:49:29 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-07-24 21:49:29 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.download